In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle as mie_params_of

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
filename = '../../computed_files/phase_equilibria_fluid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp}

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)


In [5]:
# Reading data from NIST
filename = 'nitrogen.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)


In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_fluid,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_hvap':1.0,
          'add_critical_point': True,    
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 0.2418000291208741
1 True nan
2 True 0.0544845167859296
3 True nan
4 True 0.0019330717012303
5 True nan
6 True 0.0262685212513091
7 True nan
8 True 0.007532918153911
9 True nan
10 True 0.0019330706107313
11 True nan
12 True 0.0279283948594391
13 True nan
14 True 0.0873631947291409
15 True nan
16 True 0.3197589000183619
17 True nan
18 True 0.0167794930565775
19 True nan
20 True 0.0345822220191048
21 True nan
22 True 0.0223047189511703
23 True nan
24 True 0.0018940085304394
25 True nan
26 True 0.0810899562668455
27 True nan
28 True 0.1216692491779654
29 True nan
30 True 0.001894008529733
31 True nan


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
30,30,3.03125,106.25,15.90625,True,3.658890,115.193116,16.295252,0.001894,True
24,24,2.65625,131.25,18.03125,True,3.658891,115.193249,16.295297,0.001894,True
10,10,3.43750,112.50,20.68750,True,3.662379,116.661854,16.809806,0.001933,True
4,4,2.87500,175.00,19.62500,True,3.662363,116.661868,16.809792,0.001933,True
8,8,2.68750,162.50,24.93750,True,3.699035,133.331365,24.879242,0.007533,True
18,18,3.34375,143.75,12.71875,True,3.691885,106.865998,14.231391,0.016779,False
22,22,3.21875,168.75,19.09375,True,3.767130,125.000000,19.800926,0.022305,False
6,6,3.12500,125.00,23.87500,True,3.616898,124.537037,20.780093,0.026269,False
12,12,2.81250,137.50,14.31250,True,3.542824,105.826903,13.593930,0.027928,False
20,20,2.96875,118.75,23.34375,True,3.513021,124.687500,19.453125,0.034582,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[3.65889023899925, 115.1931159655914, 16.295251872595262]